### Create training dataset for anomaly detection model
In this notebook We are going to create training dataset from node embeddings feature group and register to Hopsworks Feature Store. 
![Training Dataset](./images/create_training_dataset.png)

### Create a connection to hsfs

In [1]:
import hsfs
from hops import hdfs
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
11,application_1627907706936_0012,pyspark,idle,Link,Link


SparkSession available as 'spark'.
Connected. Call `.close()` to terminate connection gracefully.

### Retrieve alert nodes feature group from hsfs

In [2]:
alert_nodes_fg = fs.get_feature_group("alert_nodes_fg", 1)
node_embeddings_fg = fs.get_feature_group("node_embeddings_fg", 1) 

### Prepare training datasets for anomaly detection 
###### In the next notebook we are going to train [gan for anomaly detection](https://arxiv.org/pdf/1905.11034.pdf). Durring training step  we will provide only features of accounts that have never been reported for money laundering behaviour.  But we will disclose previously reported accounts to the model only in evaluation step.   

In [3]:
non_sar_emb_query = node_embeddings_fg.select(["embedding"]).join(alert_nodes_fg.select(["is_sar"]).filter(alert_nodes_fg.is_sar == 0))

In [4]:
non_sar_emb_query.show(5)

+--------------------+------+
|           embedding|is_sar|
+--------------------+------+
|[0.61500167846679...|     0|
|[0.96905446052551...|     0|
|[-0.2772440910339...|     0|
|[0.12732839584350...|     0|
|[0.95546698570251...|     0|
+--------------------+------+
only showing top 5 rows

In [5]:
non_sar_emb_query.read().count()

6531

In [6]:
non_sar_td = fs.create_training_dataset(name="gan_non_sar_training_df",
                                       version=1,
                                       data_format="tfrecord",
                                       label=["is_sar"], 
                                       statistics_config=False, 
                                       splits={'train': 0.8, 'test': 0.2},
                                       coalesce=True,
                                       description="non sar dataset for gan training")
non_sar_td.save(non_sar_emb_query)

## For testing and evaluation we will include known SAR nodes to measure anomaly score  

In [7]:
non_sar_td = fs.get_training_dataset("gan_non_sar_training_df", 1)
non_sar_test_df = non_sar_td.read(split="test")

In [8]:
sar_emb_query = node_embeddings_fg.select(["embedding"]).join(alert_nodes_fg.select(["is_sar"]).filter(alert_nodes_fg.is_sar == 1))
sar_df = sar_emb_query.read()
eval_df = non_sar_test_df.union(sar_df)
eval_df.cache()
eval_df.show()

+--------------------+------+
|           embedding|is_sar|
+--------------------+------+
|[-0.9993436336517...|     0|
|[-0.9990859031677...|     0|
|[-0.9967405796051...|     0|
|[-0.9964561462402...|     0|
|[-0.9960837364196...|     0|
|[-0.9958169460296...|     0|
|[-0.9916031360626...|     0|
|[-0.9903645515441...|     0|
|[-0.9896171092987...|     0|
|[-0.9894106388092...|     0|
|[-0.9859898090362...|     0|
|[-0.9856464862823...|     0|
|[-0.9833369255065...|     0|
|[-0.9830441474914...|     0|
|[-0.9827182292938...|     0|
|[-0.9826860427856...|     0|
|[-0.9805066585540...|     0|
|[-0.9790811538696...|     0|
|[-0.9775526523590...|     0|
|[-0.9768178462982...|     0|
+--------------------+------+
only showing top 20 rows

In [9]:
non_sar_test_df.count()

1345

In [10]:
sar_df.count()

816

In [11]:
eval_df.count()

2161

In [12]:
gan_eval_ds = fs.create_training_dataset(name="gan_eval_df",
                                       version=1,
                                       data_format="tfrecord",
                                       label=["is_sar"], 
                                       statistics_config=False, 
                                       coalesce = True,
                                       description="evaluation dataset for gan training")
gan_eval_ds.save(eval_df)